Unsloth 是一个开源工具，专门用来加速大语言模型（LLMs）的微调过程。它的主要功能和优势包括：

高效微调：Unsloth 的微调速度比传统方法快 2-5 倍，内存占用减少 50%-80%。这意味着你可以用更少的资源完成微调任务。
低显存需求：即使是消费级 GPU（如 RTX 3090），也能轻松运行 Unsloth。例如，仅需 7GB 显存就可以训练 1.5B 参数的模型。
支持多种模型和量化：Unsloth 支持 Llama、Mistral、Phi、Gemma 等主流模型，并且通过动态 4-bit 量化技术，显著降低显存占用，同时几乎不损失模型精度。
开源与免费：Unsloth 提供免费的 Colab Notebook，用户只需添加数据集并运行代码即可完成微调。

简单来说，Unsloth 采用了某些优化技术，可以帮助我们在比较低级的硬件设备资源下更高效的微调模型。在 Unsloth 出现之前，模型微调的成本非常高，普通人根本就别想了，微调一次模型至少需要几万元，几天的时间才能完成

# 实战：使用 unsloth 微调算命大师模型

## 第一步：创建环境、安装依赖

In [2]:
#这是一个jupyter notebook的魔法命令，用于隐藏命令的输出，让笔记本界面更简洁
#%%capture

!pip install unsloth

#!pip uninstall unsloth -y && pip install -upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

!pip install bitsandbytes unsloth_zoo


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 6.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/5

## 第二步：加载预训练模型

In [3]:
#导入FastLanguageModel类，用来加载和使用模型
from unsloth import FastLanguageModel
#导入torch工具，用于处理模型的数学运算
import torch

max_seq_length = 2048 #设置模型处理文本的最大长度，相当于给模型设置一个 最大容量
dtype = None #设置数据类型，让模型自动选择最适合的精度
load_in_4bit = True  #使用4位量化来节省内存，就像把大箱子压缩成小箱子

#4bit 量化（4bit Quantization）：一种技术，通过减少模型的精度来节省内存，就像把一个大箱子压缩成一个小箱子，方便携带

#加载预训练模型，并获取tokenizer工具
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B", # 指定加载的模型名称
    max_seq_length = max_seq_length, #使用最大长度
    dtype = dtype,
    load_in_4bit = load_in_4bit, #使用4位量化
    # token = "hf_...", # 如果需要访问授权模型，可以在这配置密钥
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.5: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

## 第三步：微调前测试

In [4]:
prompt_style = """以上是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 问题:
{}

### 回答:
<think>{}"""
#定义提示风格的字符串模板，用于格式化问题

question = "1986年九月初九壬申时生人，男，想了解健康运势"

In [5]:
 # 准备模型以进行推理
FastLanguageModel.for_inference(model)
# 使用tokenizer 对格式化后的问题进行编码，并移到到gpu
inputs = tokenizer([prompt_style.format(question,"")], return_tensors="pt").to("cuda")

#使用模型生成回答
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

#解码模型生成的输出为可读文本
response = tokenizer.batch_decode(outputs)

#打印生成的回答部分
print(response[0])

<｜begin▁of▁sentence｜>以上是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 问题:
1986年九月初九壬申时生人，男，想了解健康运势

### 回答:
<think>
首先，我需要了解用户的基本信息。用户是在1986年9月9日壬申时出生的男性，想要了解自己的健康运势。为了分析他的健康运势，我需要参考中国传统历法中的八卦、五行以及月份和日期的特点。

首先，分析他的生日。1986年是农历的年，但需要确定具体的生肖。1986年是兔年，因此他的生肖是兔。壬是兔的正印，所以他的命格属于壬兔。

接下来，考虑他的五行。传统上，五行与生肖有关系。兔年通常与木元素相关，所以他的五行可能与木有关。木代表长寿、健康和稳定，但也可能与肝脏、肺部等有关联。

然后，分析他的出生日期。9月9日，属于九月，属于白露之后。白露是二十四节气之一，通常与健康有关联。九月本身，天气转凉，湿气较多，容易影响人体的防卫系统。

再来看具体的健康运势。由于他的五行是木，可能需要注意肝脏和肺部的健康。传统上，木元素可能与肝脏和肺部相关，因此他可能需要关注这些部位的健康状况，避免因肝脏或肺部问题而影响整体健康。

另外，九月的气候特点也是需要考虑的因素。湿气较多，容易导致体质偏寒或湿痰。因此，他可能需要注意保暖，避免感冒，适当进行适度的运动，增强体质。

总结一下，用户的健康运势主要与木五行和九月的气候特点相关。他可能需要关注肝脏和肺部的健康，适当调理身体，保持良好的生活习惯，以维持整体健康。

在给出具体建议时，我会建议他定期体检，关注肝肺功能，注意保暖，保持心情舒畅，避免过度劳累，适当进行适度的运动，如散步、瑜伽等，以增强体质。同时，可以咨询中医师或健康专家，制定适合自己的健康计划。

通过以上分析，我为用户提供了一个全面的健康运势解读和建议，帮助他更好地了解自己，提升健康水平。
</think>

根据您的出生日期和五行特征，以下是您的健康运势分析和建议：

### 健康运势分析：
1. **五行特征**：您出生于兔年，因此五行为木。木五行代表长寿、稳重和智慧，但也与肝脏、肺部和骨骼健康有关系。因此，您需要关注这些方面的健康状况。

2. **出生日期特点**：您出

## 第四步：加载数据集

In [6]:
train_prompt_style = """以上是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是一位精通八字算命、紫微斗数、风水、易经贸卦够象、塔罗牌占卜、星象、面相手相和运势预测等方面的算命大师。
请回答以下算命问题。

###问题：
{}

##回答：
<思考>
{}
</思考>
{}"""

In [7]:
#定义结束标记(EOS_TOKEN),用于指示文本的结束
EOS_TOKEN = tokenizer.eos_token

#导入数据集加载函数
from datasets import load_dataset
#加载指定的数据集，选择中文语言和训练集的前500条记录
dataset = load_dataset ("Conard/fortune-telling", 'default', split = "train[0:200]", trust_remote_code=True)

print(dataset.column_names)


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

all_details.json:   0%|          | 0.00/693k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/207 [00:00<?, ? examples/s]

['Question', 'Response', 'Complex_CoT']


In [8]:
#定义一个函数，用于格式化数据集中的每条记录
def formatting_prompts_func(examples):
  inputs = examples["Question"]
  cots = examples["Complex_CoT"]
  outputs = examples["Response"]
  texts = [] # 格式化后的文本
  #遍历每个问题，思考过程和回答，进行格式化
  for input, cot, output in zip(inputs, cots, outputs):
    # 使用字符串模板插入数据，并加上结束标记
    text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
    texts.append(text)
  return {
      "text": texts,
  }

dataset = dataset.map(formatting_prompts_func, batched = True)
dataset["text"][0]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

'以上是描述任务的指令，以及提供进一步上下文的输入。\n请写出一个适当完成请求的回答。\n在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。\n\n### 指令：\n你是一位精通八字算命、紫微斗数、风水、易经贸卦够象、塔罗牌占卜、星象、面相手相和运势预测等方面的算命大师。\n请回答以下算命问题。\n\n###问题：\n新房装修,大门对着电梯好不好?要如何化解?\n\n##回答：\n<思考>\n好的，用户问的是新房装修时大门对着电梯好不好，以及如何化解。首先，我需要回忆一下风水学中关于大门和电梯的相关知识。电梯在风水中属于动气比较强的地方，因为电梯频繁开合，会带来不稳定的气流，也就是所谓的“煞气”。大门是住宅的纳气口，如果正对电梯，可能会让这些不稳定的气流直接冲进家里，影响居住者的健康和财运。\n\n接下来，我需要确认用户的具体情况。比如，大门和电梯的距离有多远？是否正对还是稍微偏一点？不过用户没有提供这些细节，所以只能给出一般性的建议。化解的方法通常有几种：屏风或玄关、门帘、五帝钱、植物、八卦镜等。需要逐一解释这些方法的原理和使用方式，同时提醒用户要根据实际情况选择，必要时咨询专业风水师。\n\n另外，还要注意语气要亲切，避免使用过于专业的术语，让用户容易理解。同时，要强调这些是传统方法，效果因人而异，保持客观中立。最后，可以建议用户如果情况复杂，最好请专业人士实地查看，这样更稳妥。\n\n</思考>\n根据传统风水学的观点，大门正对电梯易形成"开口煞"，电梯频繁升降会扰乱家宅气场。建议化解方案：\n\n1. 玄关阻隔法\n在入门处设置L型屏风或文化砖玄关墙，高度以1.8米为宜，既保持采光又形成缓冲带\n\n2. 五行通关法\n门槛石下埋设五帝钱+白玉葫芦，建议选丙申年铸造的真品古币，配合门楣悬挂九宫八卦镜\n\n3. 光影化解术\n安装磨砂玻璃内推门，门框镶嵌黄铜门槛，每日辰时用海盐净化门廊区域\n\n4. 现代科技方案\n入户区安装智能感应灯带，设置循环播放的流水声效，运用声光电技术平衡磁场\n\n需注意电梯井方位与家主命卦的关系，建议提供具体户型平面图进行吉凶方位测算。当代建筑中可采用半透明艺术隔断结合空气净化系统，既符合科学原理又兼顾传统智慧。<｜end▁of▁sentence｜>'

## 第五步：执行微调
*   学习率（Learning Rate）：通过 TrainingArguments 中的 learning_rate 参数设置的，这里的值为 2e-4（即 0.0002）。

*   批量大小（Batch Size）：由两个参数共同决定（实际的批量大小：per_device_train_batch_size * gradient_accumulation_steps，也就是 2 * 4 = 8）：

  *     per_device_train_batch_size：每个设备（如 GPU）上的批量大小。
  *   gradient_accumulation_steps：梯度累积步数，用于模拟更大的批量大小。

*   训练轮数（Epochs）：通过 max_steps(最大训练步数) 和数据集大小计算得出，在这段代码中，最大训练 70 步，每一步训练 8 个，数据集大小为 200，那训练论数就是 70 * 8 / 200 = 3

In [10]:
#通过 LoRA 技术对预训练模型进行了微调准备，使其能够在特定任务上进行高效的训练，同时保留预训练模型的大部分知识
FastLanguageModel.for_training(model)

model = FastLanguageModel.get_peft_model(
    model, #传入已经加载好的预训练模型
    r = 6, #设置LoRA的秩，决定添加的可训练参数数据
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_pro", "down_proj"], #指定模型中需要微调的关键模块
    lora_alpha = 16, #设置LoRA的超参数，影响可训练参数的训练方式
    lora_dropout=0, #设置防止过拟合的参数，这里设置为0表示不丢弃任何参数
    bias = "none", #设置是否添加偏置项，这里设置为none表示不添加
    use_gradient_checkpointing = "unsloth", #使用优化技术节省显存并支持更大的批量大小
    random_state = 3407,#设置随机种子，确保每次运行代码时模型的初始化方式相同
    use_rslora = False,#设置是否使用Rank Stabilized loRA技术，这里设置为False表示不使用
    loftq_config = None, #设置是否使用loftq技术，设置none表示不使用
)


Unsloth: You added custom modules, but Unsloth hasn't optimized for this.
Beware - your finetuning might be noticeably slower!


Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.3.5 patched 32 layers with 32 QKV layers, 32 O layers and 0 MLP layers.


In [13]:
from trl import SFTTrainer #用于监督式微调
from transformers import TrainingArguments #用于设置训练参数
from unsloth import is_bfloat16_supported #检查是否支持bfloat16数据格式

trainer = SFTTrainer(
    model=model, #传入要微调的模型
    tokenizer=tokenizer,#传入tokenizer,用于处理文本数据
    train_dataset=dataset,#传入训练数据集
    dataset_text_field="text",#指定数据集中文本字段的名称
    max_seq_length=max_seq_length,#设置最大序列长度
    dataset_num_proc=2,#设置数据处理的并行进程数
    packing=False,#是否启用打包功能，打包可以让训练更快，但可能影响效果
    args=TrainingArguments(#定义训练参数
        per_device_train_batch_size=2,#每个设置(如GPU)的批量大小
        gradient_accumulation_steps=4,#梯度累积步数，用于模拟大批次训练
        warmup_steps=5,#预热步数，训练开始时学习率逐渐增加的步数
        max_steps=75,#最大训练步数
        learning_rate=2e-4,#学习率，模型学习新知识的速度
        fp16=not is_bfloat16_supported(),#是否使用FP16格式加速训练(如果环境不支持)
        bf16=is_bfloat16_supported(),#是否使用bf16格式加速训练(如果环境支持)
        logging_steps=1,#每隔多少步记录一次训练日志
        optim="adamw_8bit",#使用的优化器，用于调整模型参数
        weight_decay=0.01,#权重衰减，防止模型过拟合
        lr_scheduler_type="linear",#学习率调度器类型，控制学习率的变化方式
        seed=3407,#随机种子，确认训练结果可复现
        output_dir="outputs", #训练结果保存的目录
        report_to="none",#是否将训练结果报告到外部工具，none不报告
    ),
)

Tokenizing to ["text"] (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 200 | Num Epochs = 3 | Total steps = 75
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 12,189,696/4,640,870,400 (0.26% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.383300
2,2.356400
3,2.526500
4,2.235500
5,2.123900
6,2.343900
7,2.056100
8,2.053400
9,1.975400
10,1.980000


## 第六步：微调后测试

In [15]:
#将模型切换至推理模式，准备回答问题
FastLanguageModel.for_inference(model)

#将问题转换成模型能理解的格式，并发送到GPU上
inputs = tokenizer([prompt_style.format(question,"")], return_tensors="pt").to("cuda")

#让模型根据问题生成回答，最多生成1200个新词
outputs = model.generate(
    input_ids=inputs.input_ids, #输入的数字序列
    attention_mask=inputs.attention_mask, #注意力遮罩，帮助模型理解哪些部分重要
    max_new_tokens=1200, #最多生成1200个词
    use_cache=True, #使用缓存加速生成
)

#将生成的回答从数字转换回文字
response = tokenizer.batch_decode(outputs)

#打印回答
print(response[0])



<｜begin▁of▁sentence｜>以上是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 问题:
1986年九月初九壬申时生人，男，想了解健康运势

### 回答:
<think>
<思考>
好的，用户提供了一个具体的生辰信息，需要根据传统风水学进行健康运势分析。首先，我需要确认生辰的具体日期和对应的生肖、八字排盘。

用户提供的日期是1986年九月初九壬申时，男性。根据农历月份对应的公历，九月初九是农历的九月初九，公历大约是10月8日左右（具体需要查证）。壬申是壬年申月，对应的八字排盘需要详细排列。申是阳火，壬是阳木，组合在日柱时，可能会形成火土相克的局面，需要注意健康方面的火土关系。

接下来，健康运势需要结合五行和流年运势。用户的生辰涉及申月和壬年，申月对应的是子午运，壬年属木，木克土，但土也与金有关，需要综合分析。流年方面，2024年是甲辰年，辰属土，木克土，可能会有压力；2025年乙巳年，巳属火，申月为子午，火与子水相生，可能带来健康机会。

在五行平衡方面，用户的八字中木和火为主，土和金为辅，需要注意土旺或火旺导致的健康问题。建议增强土的力量，如佩戴土属性饰品，或者增强水木的调和。同时，注意避免过度劳累，保持心情舒畅。

具体健康方面，可能会有脾胃或肝胆方面的问题，建议多吃补品，注意饮食调理。流年运势方面，2024年可能有压力，2025年则有机遇，注意适时调整。

最后，总结运势时要保持积极，提醒用户注意平衡和调和，保持健康心态。可能还需要建议一些风水调整，比如家居布局或佩戴饰品来增强健康运势。

</思考>
**1986年九月初九壬申时（公历约10月8日）男性健康运势分析**

**八字排盘**  
日柱：壬申（木火）  
月柱：申（火）  
年柱：壬（木）  
时柱：申（火）  
**五行组合**：木火旺，土金为辅，需平衡火土相克。

---

### **健康运势**
1. **本命五行**  
   - **木火旺**：需注意脾胃、肝胆功能，建议增强水土调和，避免过度劳累。
   - **土金辅助**：需防肾脏、皮肤问题，注意保湿养护。

2. **流年压力**  
   - **2024甲辰年**（辰土）：木克土，可能引

## 第七步：将微调后的模型保存为GGUF格式

GGUF 是一种高效的格式，它支持多种量化方法（如 4 位、8 位、16 位量化），可以显著减小模型文件的大小，便于存储和传输，适合在资源受限的设备上运行模型，例如在 Ollama 上部署时。量化后的模型在资源受限的设备上运行更快，适合边缘设备或低功耗场景。

In [17]:
#导入google colab的 userdata模块，用于访问用户数据
from google.colab import userdata

#从google colab用户数据中获取hugging face的api令牌
HUGGINGFACE_TOKEN = userdata.get("HUGGINGFACE")

#将模型保存为8位量化格式
if True: model.save_pretrained_gguf("model", tokenizer,)

#将模型保存为16位量化格式，精度更高，但文件稍大
if False: model.save_pretrained_gguf("model_f16", tokenizer, quantization_method = "f16")

#将模型保存为4位量化格式，精度稍低，但文件最小
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")


Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.
Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.01 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 50%|█████     | 16/32 [00:01<00:01, 15.97it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [02:46<00:00,  5.22s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /content/model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: /content/model/unsloth.Q8_0.gguf


## 第八步：将微调后的模型上传到HuggingFace

In [19]:
from huggingface_hub import create_repo

#在hugging face 创建一个模型仓库
create_repo("bondzhan/fine-tuning", token=HUGGINGFACE_TOKEN, exist_ok=True)

#将模型和分词器上传到hugging face hub上的仓库
model.push_to_hub_gguf("bondzhan/fine-tuning", tokenizer, token=HUGGINGFACE_TOKEN)


Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.7 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [03:15<00:00,  6.10s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving bondzhan/fine-tuning/pytorch_model-00001-of-00004.bin...
Unsloth: Saving bondzhan/fine-tuning/pytorch_model-00002-of-00004.bin...
Unsloth: Saving bondzhan/fine-tuning/pytorch_model-00003-of-00004.bin...
Unsloth: Saving bondzhan/fine-tuning/pytorch_model-00004-of-00004.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at bondzhan/fine-tuning into q8_0 GGUF format.
The output location will be /content/bondzhan/fine-tuning/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: fine-tuning
INFO:gguf.gguf_writer:gguf: This GGUF 

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q8_0.gguf:   0%|          | 0.00/8.54G [00:00<?, ?B/s]

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Saved GGUF to https://huggingface.co/bondzhan/fine-tuning
